In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
img_path = "/kaggle/input/vehicle-classification/Vehicles"

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
class Vehicle_Dataset(Dataset):
    def __init__(self, data_dir, transform = None):
        self.data = ImageFolder(data_dir, transform = transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

    

In [ ]:
target_to_class ={}

for v,k in ImageFolder(img_path).class_to_idx.items():
    target_to_class[k] = v

In [ ]:
target_to_class

In [ ]:
import timm

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
dataset = Vehicle_Dataset("/kaggle/input/vehicle-classification/Vehicles", transform = transform)

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

In [ ]:
def load_model():
    model = timm.create_model('efficientnet_b0', pretrained = True)
    model = nn.Sequential(
    *list(model.children())[:-1],
    nn.Linear(1280,7))
    #print(model)
    return model

In [ ]:
model = load_model()

In [ ]:
def return_loader(dataset,batch_size):
    return DataLoader(
    dataset,
    batch_size = batch_size,
    shuffle=True,
    )

In [ ]:
import shutil
main_dir = "/kaggle/input/vehicle-classification/Vehicles"
categories = ["Auto Rickshaws", "Bikes", "Cars", "Motorcycles", "Planes", "Ships", "Trains"]

split_dirs = ["train", "val", "test"]
base_split_dir = "/kaggle/working/vehicles_split"

for split_dir in split_dirs:
    for category in categories:
        os.makedirs(os.path.join(base_split_dir, split_dir, category), exist_ok=True)
        
for category in categories:
    category_path = os.path.join(main_dir, category)
    images = os.listdir(category_path)
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    def move_images(image_list, destination):
        for image in image_list:
            shutil.copy(os.path.join(category_path, image), os.path.join(base_split_dir, destination, category, image))

    move_images(train_images, "train")
    move_images(val_images, "val")
    move_images(test_images, "test")

In [ ]:
train_dataset = Vehicle_Dataset("/kaggle/working/vehicles_split/train", transform = transform)
val_dataset = Vehicle_Dataset("/kaggle/working/vehicles_split/val", transform = transform)
model = load_model()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
train_loader = return_loader(train_dataset,64)
valid_loader = return_loader(val_dataset,64)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
max_epochs = 20
save_every = 5
training_losses = []
valid_losses = []
model.to(device)
for epoch in range(max_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc= "Training Loop"):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * labels.size(0)
    
    train_loss = running_loss/len(train_loader)
    training_losses.append(train_loss)
    
    model.eval()
    runnning_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc= "Validation Loop"):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            running_loss += loss.item() * labels.size(0)
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss)
    
    
    print(f"Epoch :{epoch+1}  | Training_Loss :{train_loss}  | Validation loss :{valid_loss}")
    if(epoch%save_every == 0):
        chk = model.state_dict()
        torch.save(chk, "/kaggle/working/Checkpoint.pt")
        print(f"Epochs :{epoch+1} | Training saved successfully! ")

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(1,2)
axis[0].plot(training_losses)

axis[1].plot(valid_losses)
plt.show()

In [ ]:
test_dataset = Vehicle_Dataset("/kaggle/working/vehicles_split/test", transform = transform)
test_loader = return_loader(test_dataset, 64)

In [ ]:
def compute_accuracy(y_pred, y_true):
    acc = np.sum(y_pred == y_true)/y_true.shape[0]
    return acc

In [ ]:
accuracies= []
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        output = output.detach().cpu().numpy()
        output = np.argmax(output, axis = 1)
        labels = labels.detach().cpu().numpy()
        accuracy = compute_accuracy(output, labels)
        accuracies.append(accuracy)

In [ ]:
accuracy_avg = sum(accuracies)/len(accuracies)

In [ ]:
print(f"Accuracy for test set is : {accuracy_avg *100} %")